# Nested Chats for Solving Complex Tasks with a Sequence of Inner Monologue

This notebook showcases how to use nested chats in AutoGen to to solving complex task with one or a sequence chats nested.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

### Example Tasks

In [1]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 44}

tasks = [
    """On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.""",
    """Investigate possible reasons for the performance.""",
]

Neither powershell nor pwsh is installed.


In [2]:
inner_assistant = autogen.AssistantAgent(
    "Inner-assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

inner_code_interpreter = autogen.UserProxyAgent(
    "Inner-code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat = autogen.GroupChat(
    agents=[inner_assistant, inner_code_interpreter],
    messages=[],
    speaker_selection_method="round_robin",  # With two agents, this is equivalent to a 1:1 conversation.
    allow_repeat_speaker=False,
    max_round=8,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [3]:
assistant_1 = autogen.AssistantAgent(
    name="Assistant_1",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)

assistant_2 = autogen.AssistantAgent(
    name="Assistant_2",
    llm_config={"config_list": config_list},
    # is_termination_msg=lambda x: x.get("content", "") == "",
)
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    """,
)

reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a compliance reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    You must review carefully, identify potential issues, and maintain the integrity of the organization.
    Your role demands fairness, a deep understanding of regulations, and a focus on protecting against
    harm while upholding a culture of responsibility.
    """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [4]:
def writing_message(recipient, messages, sender, config):
    return f"Polish the content to make an engaging and nicely formatted blog post. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {"recipient": manager, "summary_method": "reflection_with_llm"},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": reviewer, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]
assistant_1.register_nested_chats(
    [autogen.Agent, None],
    nested_chat_queue,
)
# user.initiate_chat(assistant, message=tasks[0], max_turns=1)

user.initiate_chats(
    [
        {"recipient": assistant_1, "message": tasks[0], "max_turns": 1, "summary_method": "last_msg"},
        {"recipient": assistant_2, "message": tasks[1]},
    ]
)


********************************************************************************
Start a new chat with the following message: 
On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

With the following carryover: 


********************************************************************************
User (to Assistant_1):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.

With the following carryover: 


********************************************************************************
Assistant_1 (to chat_manager):

On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.


/Users/qingyunwu/Documents/github/autogen/autogen/agentchat/chat.py:79: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...
Inner-code-interpreter (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

Date: 2024-01-24, Close: $401.82
Date: 2024-01-25, Close: $404.12
Date: 2024-01-26, Close: $403.18
Date: 2024-01-29, Close: $408.96
Date: 2024-01-30, Close: $407.84
Date: 2024-02-01, Close: $403.03
Date: 2024-02-02, Close: $410.46
Date: 2024-02-05, Close: $404.90
Date: 2024-02-06, Close: $404.74
Date: 2024-02-07, Close: $413.29
Date: 2024-02-08, Close: $413.35
Date: 2024-02-09, Close: $419.77
Date: 2024-02-12, Close: $414.49
Date: 2024-02-13, Close: $405.57
Date: 2024-02-14, Close: $409.49
Date: 2024-02-15, Close: $406.56
Date: 2024-02-16, Close: $404.06
Date: 2024-02-20, Close: $402.79

Microsoft stock was higher than $400 on 18 days between 2024-01-24 and 2024-02-20 in 2024.


--------------------------------------------------------------------------------
Inner-assistant (to chat_manager):

Based on the stock data retrieved, Mi

[ChatResult(chat_history=[{'content': 'On which days in 2024 was Microsoft Stock higher than $400? Comment on the stock performance.', 'role': 'assistant'}, {'content': "# Microsoft's High-Flying Start to 2024: Unpacking the Stock's Soar Above $400\n\nIn the frosty embrace of early 2024, Microsoft Corp.'s stock price emerged not merely as a figure on screens but as a prevailing narrative of growth, triumphing over the hallowed $400 threshold on 18 noteworthy days from January 24 to February 20. This remarkable trend in Microsoft's financial journey reflects a vigor within the tech behemoth that caught the eyes of investors and marked a roaring start to the year.\n\n## Peaks of Prosperity: The Journey Beyond $400\n\nThe financial markets, akin to an ever-watching audience, witnessed Microsoft's shares take flight, each venture past the $400 mark underscoring the firm's towering presence and industry stamina. Not content with mere flirtation, Microsoft's shares danced above and beyond, a